In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
news = pd.read_csv('/content/drive/MyDrive/datasets/trainset.txt',sep='\t', names = ['CLASS','TITLE','DATE','BODY'])
news.head()

,CLASS,TITLE,DATE,BODY
0,1,JAPAN FIRM PLANS TO SELL U.S. FARMLAND TO JAP...,"MORIOKA, Japan, March 12 -",A Japanese real estate company said it will ...
1,-1,NORTH BH SETS ONE-FOR-FIVE OFFER FOR NORGOLD ...,"MELBOURNE, March 12 -",North Broken Hill Holdings Ltd & lt;NBHA.ME ...
2,-1,OUTOKUMPU IN COPPER DEAL WITH IBERICA DEL COBRE,"HELSINKI, June 26 -",Finland's state-owned mining company Outokum...
3,1,ROTTERDAM GRAIN HANDLER SAYS PORT BALANCE ROSE,"ROTTERDAM, April 13 -","Graan Elevator Mij, GEM, said its balance in..."
4,1,U.S. SENATE PANEL VOTES TO LIMIT COUNTY LOAN ...,(NO DATE),(NO TEXT)


In [ ]:
#data preparation
texts = news['BODY']
labels = news['CLASS']


In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Load data
import pandas as pd
news = pd.read_csv('/content/drive/MyDrive/datasets/trainset.txt', sep='\t', names=['CLASS', 'TITLE', 'DATE', 'BODY'])

# Define a function for text preprocessing
def preprocess_text(text):
    # Remove non-alphabetic characters
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Apply text preprocessing to the 'BODY' column
news['BODY'] = news['BODY'].apply(preprocess_text)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Represent the dataset as a matrix format
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(news['BODY']).toarray()


In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, news['CLASS'], test_size=0.2, random_state=42)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Choose K Nearest Neighbor
knn_classifier = KNeighborsClassifier(n_neighbors=10)
knn_classifier.fit(X_train, y_train)

# Evaluate accuracy on the test set
accuracy = knn_classifier.score(X_test, y_test)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.60


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Use TF-IDF instead of CountVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(news['BODY']).toarray()

# Split the dataset
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, news['CLASS'], test_size=0.2, random_state=42)

# Choose a different classifier (Random Forest) and perform hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
}

rf_classifier = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train_tfidf)

# Get the best parameters and evaluate on the test set
best_params = grid_search.best_params_
best_rf_classifier = grid_search.best_estimator_

accuracy_tfidf = best_rf_classifier.score(X_test_tfidf, y_test_tfidf)
print(f"Accuracy with TF-IDF and Random Forest: {accuracy_tfidf:.2f}")


Accuracy with TF-IDF and Random Forest: 0.95


In [ ]:
import pandas as pd
testdata = pd.read_csv('/content/drive/MyDrive/datasets/testsetwithoutlabels.txt',sep='\t', names = ['TITLE','DATE','BODY'])
testdata.head()
# Define a function for text preprocessing


,TITLE,DATE,BODY
0,USSR WHEAT BONUS OFFER SAID STILL UNDER DEBATE,"WASHINGTON, March 6 -",The Reagan administration continues to debat...
1,HAITIAN CANE PLANTERS PROTEST SUGAR MILL CLOSURE,"PORT-AU-PRINCE, April 13 -","About 2,000 sugar cane planters marched to P..."
2,U.S. URGES JAPAN TO OPEN FARM MARKET FURTHER,"TOKYO, April 17 -",U.S. Agriculture Secretary Richard Lyng has ...
3,NORANDA COPPER MINE SHUT DOWN AFTER FIRE,"TORONTO, April 2 -",Noranda Inc said production at its Murdochvi...
4,17 MISSING NORANDA MINERS FOUND ALIVE,"MURDOCHVILLE, Que., April 2-",Seventeen miners who were missing in a coppe...


In [ ]:
testdata['BODY2'] = testdata['BODY'].apply(preprocess_text)
testdata.head(2)

,TITLE,DATE,BODY,BODY2
0,USSR WHEAT BONUS OFFER SAID STILL UNDER DEBATE,"WASHINGTON, March 6 -",The Reagan administration continues to debat...,reagan administration continues debate whether...
1,HAITIAN CANE PLANTERS PROTEST SUGAR MILL CLOSURE,"PORT-AU-PRINCE, April 13 -","About 2,000 sugar cane planters marched to P...",sugar cane planters marched port au prince pro...


In [ ]:
Xtest = vectorizer.transform(testdata['BODY2'])
print(Xtest.toarray())
print(X.shape)
print(Xtest.shape)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(200, 4249)
(100, 4249)


In [ ]:
testdata['BODY2'] = testdata['BODY'].apply(preprocess_text)

# Represent the test dataset as a TF-IDF matrix using the same vectorizer
X_test_tfidf = tfidf_vectorizer.transform(testdata['BODY2']).toarray()

# Make predictions using the trained Random Forest classifier
test_predictions = best_rf_classifier.predict(X_test_tfidf)

# Add predictions to the testdata DataFrame
testdata['PREDICTION'] = test_predictions

# Display the updated testdata DataFrame with predictions
print(testdata[['BODY', 'PREDICTION']])


                                                 BODY  PREDICTION
0    The Reagan administration continues  to debat...           1
1    About 2,000 sugar cane planters  marched to P...           1
2    U.S. Agriculture Secretary Richard Lyng  has ...           1
3    Noranda Inc said production at its  Murdochvi...          -1
4    Seventeen miners who were  missing in a coppe...          -1
..                                                ...         ...
95   Three black workers were killed and  seven in...          -1
96   World closing stocks of primary zinc  at smel...           1
97   Any further decline in raw sugar prices  is l...           1
98   & lt;Noranda Inc > said a copper  mine fire t...          -1
99   There were three grain ships loading  and two...           1

[100 rows x 2 columns]
